<a href="https://colab.research.google.com/github/alyssa-blair/csc-485B-group8/blob/main/csc485b_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-c3ut3oj1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-c3ut3oj1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <iostream> // for std::ostream
#include <vector>


namespace csc485b{
namespace a2{

using node_t = int;
using edge_t = int2;

using edge_list_t = std::vector< edge_t >;
using node_list_t = std::vector< node_t >;

} // namespace a2
} // namespace csc485b


In [ ]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [ ]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <cassert>  // for assert()
#include <cstddef>  // std::size_t type
#include <random>   // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a2 {

/**
 * Generates and returns a vector of random edges
 * for a graph `G=(V,E)` with `n=|V|=n` and expected `m=|E|`.
 * Referred to as an Erdős-Rényi graph.
 *
 * @see https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.fast_gnp_random_graph.html#networkx.generators.random_graphs.fast_gnp_random_graph
 */
edge_list_t generate_graph( std::size_t n, std::size_t m )
{
    assert( "At most n(n-1) edges in a simple graph" && m < n * ( n - 1 ) );

    int const probability = ( 100 * m ) / ( n * ( n - 1 ) );

    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20241008;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 100);

    edge_list_t random_edges;
    random_edges.reserve( 2 * m );

    for( node_t u = 0; u < n; ++u )
    {
        for( node_t v = u + 1; v < n; ++v )
        {
            auto const dice_roll = distrib( rng );
            if( dice_roll <= probability )
            {
                random_edges.push_back( make_int2( u, v ) );
                random_edges.push_back( make_int2( v, u ) );
            }
        }
    }

    random_edges.resize( random_edges.size() );


    return random_edges;
}

} // namespace a2
} // namespace csc485b


In [ ]:
%%cuda_group_save -g "source" -n "gemm.h"

/**
 * The file in which you will implement your DenseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type
#include <mma.h>
#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {
namespace gpu {
using namespace nvcuda;

#define TILE_SIZE 16
/**
  * Repopulates the adjacency matrix as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__device__
void GEMM_old( node_t * G, std::size_t n )
{
    const int by = blockIdx.y;
    const int bx = blockIdx.x;

    // Constants for this kernel
    const int tx = threadIdx.x;
    const int ty = threadIdx.y;

    // Global indices (for accessing global memory)
    std::size_t r = bx * blockDim.x + tx;
    std::size_t c = by * blockDim.y + ty;

    // Shared memory using tiling
    __shared__ node_t A[TILE_SIZE][TILE_SIZE];
    __shared__ node_t B[TILE_SIZE][TILE_SIZE];

    // Each thread is responsible for one value
    node_t val = 0;

    // We loop through all tiles needed
    for (int i = 0; i < n / TILE_SIZE; i++)
    {
        // Load this tile batch into shared memory
        A[ty][tx] = G[r * n + i * TILE_SIZE + tx];
        B[ty][tx] = G[(i * TILE_SIZE + ty) * n + c];
        __syncthreads();

        // Perform the partial dot product
        for (int j = 0; j < TILE_SIZE; j++)
        {
            val += A[ty][j] * B[j][tx];
        }
        __syncthreads();
    }

    // Remove the diagonal and clamp values back to [0,1]
    val = val > 0 ? 1 : 0;
    val = r == c ? 0 : val;
    G[r * n + c] = val;
}


__device__
void GEMM( half * G, float * C, std::size_t n )
{
    const int by = blockIdx.y;
    const int bx = blockIdx.x;

    // Constants for this kernel
    const int tx = threadIdx.x;
    const int ty = threadIdx.y;

    // Global indices (for accessing global memory)
    std::size_t r = bx * blockDim.x + tx;
    std::size_t c = by * blockDim.y + ty;

   int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / 32;
   int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

   wmma::fragment<wmma::matrix_a, TILE_SIZE, TILE_SIZE, TILE_SIZE, half, wmma::col_major> a_frag;
   wmma::fragment<wmma::matrix_b, TILE_SIZE, TILE_SIZE, TILE_SIZE, half, wmma::col_major> b_frag;
   wmma::fragment<wmma::accumulator, TILE_SIZE, TILE_SIZE, TILE_SIZE, float> acc_frag;
   wmma::fragment<wmma::accumulator, TILE_SIZE, TILE_SIZE, TILE_SIZE, float> c_frag;

   wmma::fill_fragment(acc_frag, 0.0f);


   for (int i = 0; i < n; i+= TILE_SIZE) {
      int aCol = i;
      int aRow = warpM * TILE_SIZE;
      int bCol = warpN * TILE_SIZE;
      int bRow = i;

       if (aRow < n && bCol < n && aCol < n && bRow < n) {
           wmma::load_matrix_sync(a_frag, G + aRow + aCol * n, n);
           wmma::load_matrix_sync(b_frag, G + bRow + bCol * n, n);

           wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
       }
   }

  // Load in current value of c, scale by beta, and add to result scaled by alpha
  int cRow = warpM * TILE_SIZE;
  int cCol = warpN * TILE_SIZE;
  int val;

  if (cRow < n && cCol < n) {
      wmma::load_matrix_sync(c_frag, C + cRow + cCol * n, n, wmma::mem_col_major);

      for(int i=0; i < c_frag.num_elements; i++) {
          val = acc_frag.x[i] + c_frag.x[i];
          val = val > 0 ? 1 : 0;
          val = r == c ? 0 : val;
          c_frag.x[i] = val;
      }

      // Store the output
      wmma::store_matrix_sync(C + cRow + cCol * n, c_frag, n, wmma::mem_col_major);
    }
}

 } // namespace gpu
} // namespace a2
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "dense_graph.h"
/**
 * The file in which you will implement your DenseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"
#include "gemm.h"

namespace csc485b {
namespace a2      {

/**
 * A DenseGraph is optimised for a graph in which the number of edges
 * is close to n(n-1). It is represented using an adjacency matrix.
 */
struct DenseGraph
{
  std::size_t n; /** Number of nodes in the graph. */
  node_t * adjacencyMatrix; /** Pointer to an n x n adj. matrix */

  /** Returns number of cells in the adjacency matrix. */
  __device__ __host__ __forceinline__
  std::size_t matrix_size() const { return n * n; }
};

namespace gpu {

/**
 * Constructs a DenseGraph from an input edge list of m edges.
 *
 * @pre The pointers in DenseGraph g have already been allocated.
 */
__global__
void build_graph( DenseGraph g, edge_t const * edge_list, std::size_t m )
{
    const std::size_t th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if (th_id < m) {
      edge_t edge = edge_list[th_id];
      g.adjacencyMatrix[edge.x * g.n + edge.y] = 1;
    }
    return;
}

//#define TILE_SIZE 4
#define TILE_SIZE 16
/**
  * Repopulates the adjacency matrix as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( DenseGraph g )
{
    // Perform GEMM operation with the half-precision matrix
    //GEMM(reinterpret_cast<half *>(g.adjacencyMatrix), reinterpret_cast<float *>(g.adjacencyMatrix), g.n);

    GEMM_old(g.adjacencyMatrix, g.n);
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <chrono>   // for timing
#include <iostream> // std::cout, std::endl
#include <iterator> // std::ostream_iterator
#include <vector>

#include "dense_graph.h"

#include "data_generator.h"
#include "data_types.h"

/**
 * Runs timing tests on a CUDA graph implementation.
 * Consists of independently constructing the graph and then
 * modifying it to its two-hop neighbourhood.
 */
template < typename DeviceGraph >
void run( DeviceGraph g, csc485b::a2::edge_t const * d_edges, std::size_t m )
{
    cudaDeviceSynchronize();
    auto const build_start = std::chrono::high_resolution_clock::now();

    std::size_t const threads_per_block = m >> 1;
    std::size_t const num_blocks = ( m + threads_per_block - 1 ) / threads_per_block;

    // this code doesn't work yet!
    csc485b::a2::gpu::build_graph<<< num_blocks, threads_per_block >>>( g, d_edges, m );

    cudaDeviceSynchronize();
    auto const reachability_start = std::chrono::high_resolution_clock::now();

    // neither does this!
    std::size_t const threads_per_block_2 = g.n;
    std::size_t const num_blocks_2 = g.n;
    csc485b::a2::gpu::two_hop_reachability<<< num_blocks_2, threads_per_block_2 >>>( g );

    cudaDeviceSynchronize();
    auto const end = std::chrono::high_resolution_clock::now();


    std::cout << "Build time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(reachability_start - build_start).count()
              << " us"
              << std::endl;

    std::cout << "Reachability time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(end - reachability_start).count()
              << " us"
              << std::endl;
}

/**
 * Allocates space for a dense graph and then runs the test code on it.
 */
void run_dense( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device DenseGraph
    a2::node_t * d_matrix;
    cudaMalloc( (void**)&d_matrix, sizeof( a2::node_t ) * n * n );
    a2::DenseGraph d_dg{ n, d_matrix };

    run( d_dg, d_edges, m );

    // check output?
    //std::vector< a2::node_t > host_matrix( d_dg.matrix_size() );
    //a2::DenseGraph dg{ n, host_matrix.data() };
    //cudaMemcpy( dg.adjacencyMatrix, d_dg.adjacencyMatrix, sizeof( a2::node_t ) * d_dg.matrix_size(), cudaMemcpyDeviceToHost );
    //std::copy( host_matrix.cbegin(), host_matrix.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );

    // clean up
    cudaFree( d_matrix );
}


int main()
{
    using namespace csc485b;

    // Create input
    std::size_t constexpr n = 512;
    //std::size_t constexpr n = 16;
    std::size_t constexpr expected_degree = n >> 1;

    a2::edge_list_t const graph = a2::generate_graph( n, n * expected_degree );
    std::size_t const m = graph.size();

    // lazily echo out input graph
    for( auto const& e : graph )
    {
         std::cout << "(" << e.x << "," << e.y << ") ";
    }

    // allocate and memcpy input to device
    a2::edge_t * d_edges;
    cudaMalloc( (void**)&d_edges, sizeof( a2::edge_t ) * m );
    cudaMemcpyAsync( d_edges, graph.data(), sizeof( a2::edge_t ) * m, cudaMemcpyHostToDevice );

    // run your code!
    run_dense ( d_edges, n, m );

    return EXIT_SUCCESS;
}

In [ ]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

(0,2) (2,0) (0,3) (3,0) (0,4) (4,0) (0,5) (5,0) (0,6) (6,0) (0,9) (9,0) (0,10) (10,0) (0,11) (11,0) (0,13) (13,0) (0,14) (14,0) (0,16) (16,0) (0,20) (20,0) (0,21) (21,0) (0,23) (23,0) (0,24) (24,0) (0,26) (26,0) (0,27) (27,0) (0,28) (28,0) (0,30) (30,0) (0,31) (31,0) (0,32) (32,0) (0,33) (33,0) (0,35) (35,0) (0,36) (36,0) (0,38) (38,0) (0,40) (40,0) (0,43) (43,0) (0,44) (44,0) (0,46) (46,0) (0,50) (50,0) (0,51) (51,0) (0,54) (54,0) (0,60) (60,0) (0,63) (63,0) (0,66) (66,0) (0,67) (67,0) (0,68) (68,0) (0,69) (69,0) (0,71) (71,0) (0,74) (74,0) (0,75) (75,0) (0,76) (76,0) (0,79) (79,0) (0,81) (81,0) (0,83) (83,0) (0,85) (85,0) (0,87) (87,0) (0,89) (89,0) (0,90) (90,0) (0,91) (91,0) (0,92) (92,0) (0,93) (93,0) (0,95) (95,0) (0,100) (100,0) (0,101) (101,0) (0,103) (103,0) (0,104) (104,0) (0,106) (106,0) (0,107) (107,0) (0,108) (108,0) (0,109) (109,0) (0,111) (111,0) (0,113) (113,0) (0,114) (114,0) (0,117) (117,0) (0,118) (118,0) (0,119) (119,0) (0,120) (120,0) (0,121) (121,0) (0,126) (126,0